In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt

In [ ]:
# đường dẫn đến folder chứa dataset
dataset_path = '/content/drive/MyDrive/Projects/Computer Vision/Face mask detection/Dataset/'
xml_path = os.path.join(dataset_path, 'annotations')
img_path = os.path.join(dataset_path, 'images')

In [ ]:
def display_image(image, cropped_image):

  plt.figure(figsize=(10, 5))

  plt.subplot(1, 2, 1)
  plt.title('ảnh gốc')
  plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
  plt.axis('off')

  plt.subplot(1, 2, 2)
  plt.title('ảnh cắt')
  plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
  plt.axis('off')

  plt.show()

In [ ]:
def image_exists(img_path):
  return os.path.isfile(img_path)

In [ ]:
def process_xml(xml_file, save_dir, count, img_path):

  # Đọc file XML
  tree = ET.parse(xml_file)
  root = tree.getroot()

  # Lấy đường dẫn thư mục chứa file XML
  xml_dir = os.path.dirname(xml_file)

  # Lặp qua tất cả các đối tượng trong file XML
  for obj in root.findall('object'):
    # Lấy tên đối tượng từ mục name
    name = obj.find('name').text

    # Lấy thông tin bounding box từ mỗi đối tượng
    xmin = int(obj.find('.//xmin').text)
    ymin = int(obj.find('.//ymin').text)
    xmax = int(obj.find('.//xmax').text)
    ymax = int(obj.find('.//ymax').text)

    # Đường dẫn ảnh từ XML
    img_filename = root.find('filename').text
    img_dir = os.path.join(img_path, img_filename)

    # Kiểm tra xem file ảnh tồn tại không
    if img_dir and image_exists(img_dir):
      image = cv2.imread(img_dir)

      # Kiểm tra xem ảnh đã đọc thành công hay không
      if image is not None:
        # Cắt ảnh
        cropped_image = image[ymin:ymax, xmin:xmax]

        # Tạo thư mục nếu chưa tồn tại
        obj_dir = os.path.join(save_dir, name)
        if not os.path.exists(obj_dir):
          os.makedirs(obj_dir)

        # Tạo tên cho file ảnh đã cắt
        output_filename = f"{str(count).zfill(4)}_{name}.jpg"
        output_path = os.path.join(obj_dir, output_filename)

        count += 1

        # Hiển thị ảnh gốc và ảnh cắt
        display_image(image, cropped_image)

        # Lưu ảnh đã cắt
        cv2.imwrite(output_path, cropped_image)

        print(f"Đã lưu {output_filename} vào thư mục {obj_dir}")
      else:
        print(f"Không thể đọc ảnh {img_dir} - Bỏ qua...")
    else:
      print(f"Không tìm thấy file ảnh cho {img_filename} - Bỏ qua...")

  return count

In [ ]:
# Thư mục chứa ảnh cắt của tập train và test
train_save_dir = os.path.join(dataset_path, 'data_cut')

count = 1

# Xử lý tập train
for xml_file in os.listdir(xml_path):
    if xml_file.endswith(".xml"):
        xml_dir = os.path.join(xml_path, xml_file)
        count = process_xml(xml_dir, train_save_dir, count, img_path)
        os.remove(xml_dir)
